In [1]:
from src.satellite_images.storage import SentinelDataset

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
from matplotlib import pyplot as plt

plt.style.use("ggplot")

CLASSES = ['bygg', 'rug', 'rughvete', 'hvete', 'havre']

In [2]:
import geopandas as gpd


sd = SentinelDataset('E:/MasterThesisData/Satellite_Images/small_images_all.h5')


In [63]:
labels_lookup = gpd.read_file('../../kornmo-data-files/raw-data/crop-classification-data/all_data.gpkg')
labels_lookup["key"] = labels_lookup.orgnr.astype(int).astype(str) + labels_lookup.index.astype(str) + "/" + labels_lookup.year.astype(int).astype(str)
labels_lookup.drop(columns=["year", "orgnr"], inplace=True)

labels_lookup.drop(labels_lookup[labels_lookup['area'] < 1500].index, inplace = True)
labels_lookup = labels_lookup.loc[labels_lookup['planted'] != 'erter']
labels_lookup = labels_lookup.loc[labels_lookup['planted'] != 'oljefro']

# labels_lookup = labels_lookup.set_index("key")
labels_lookup

,planted,area,geometry,key
0,bygg,94540.480524,"POLYGON ((301788.79090 6608202.23000, 301789.4...",8119356620/2019
1,bygg,22778.560014,"POLYGON ((301588.81297 6607862.03284, 301582.7...",8119356621/2019
2,bygg,5072.924216,"POLYGON ((301544.54510 6608141.21000, 301539.9...",8119356622/2019
3,bygg,11611.972746,"POLYGON ((301285.90120 6608685.32060, 301285.7...",8119356623/2019
4,havre,10203.282317,"POLYGON ((281369.42428 6687420.41295, 281369.5...",8125305424/2019
...,...,...,...,...
404939,None,81170.744027,"POLYGON ((292000.79988 6740131.57158, 291935.5...",999315089404939/2017
404940,None,47871.002364,"POLYGON ((292036.72990 6740255.23060, 292031.3...",999315089404940/2017
404941,None,22108.550434,"POLYGON ((326429.63114 7113387.26610, 326330.0...",999552579404941/2017
404942,None,65288.461984,"POLYGON ((326242.48690 7113421.74470, 326235.8...",999552579404942/2017


In [64]:
labels_to_keep = [f"{int(label.split('/')[1])}/{int(label.split('/')[2])}" for label in sd.labels]

print(pd.Series(list(labels_lookup['planted'])).value_counts())
labels_lookup

bygg        76231
havre       20512
hvete       18102
rug           816
rughvete      517
dtype: int64


,planted,area,geometry,key
0,bygg,94540.480524,"POLYGON ((301788.79090 6608202.23000, 301789.4...",8119356620/2019
1,bygg,22778.560014,"POLYGON ((301588.81297 6607862.03284, 301582.7...",8119356621/2019
2,bygg,5072.924216,"POLYGON ((301544.54510 6608141.21000, 301539.9...",8119356622/2019
3,bygg,11611.972746,"POLYGON ((301285.90120 6608685.32060, 301285.7...",8119356623/2019
4,havre,10203.282317,"POLYGON ((281369.42428 6687420.41295, 281369.5...",8125305424/2019
...,...,...,...,...
404939,None,81170.744027,"POLYGON ((292000.79988 6740131.57158, 291935.5...",999315089404939/2017
404940,None,47871.002364,"POLYGON ((292036.72990 6740255.23060, 292031.3...",999315089404940/2017
404941,None,22108.550434,"POLYGON ((326429.63114 7113387.26610, 326330.0...",999552579404941/2017
404942,None,65288.461984,"POLYGON ((326242.48690 7113421.74470, 326235.8...",999552579404942/2017


In [65]:


# bygg = labels_lookup.loc[(labels_lookup['planted'].isnull()) | (labels_lookup['planted'] == 'bygg')]
# rug = labels_lookup.loc[(labels_lookup['planted'].isnull()) | (labels_lookup['planted'] == 'rug')]
# rughvete = labels_lookup.loc[(labels_lookup['planted'].isnull()) | (labels_lookup['planted'] == 'rughvete')]
# hvete = labels_lookup.loc[(labels_lookup['planted'].isnull()) | (labels_lookup['planted'] == 'hvete')]
# havre = labels_lookup.loc[(labels_lookup['planted'].isnull()) | (labels_lookup['planted'] == 'havre')]

# print(f'Bygg {bygg.shape}')
# print(f'Rug {rug.shape}')
# print(f'Rughvete {rughvete.shape}')
# print(f'Hvete {hvete.shape}')
# print(f'Havre {havre.shape}')


In [105]:
BAG_COUNT = 3500
VAL_BAG_COUNT = 500
BAG_SIZE = 7
PLOT_SIZE = 3

In [95]:
from sklearn.model_selection import train_test_split
labels = labels_lookup['planted']

data_x = []
data_y = []
for i, label in tqdm(enumerate(labels_to_keep), total=len(labels_to_keep)):
    data_x.append(label)
    data_y.append(labels.iloc[i])


x_train, x_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.3, random_state=42)
print(f'x_train: {len(x_train)}')
print(f'x_val: {len(x_val)}')
print(f'y_train: {len(y_train)}')
print(f'y_val: {len(y_val)}')

100%|██████████| 178390/178390 [00:00<00:00, 292467.59it/s]


x_train: 124873
x_val: 53517
y_train: 124873
y_val: 53517


In [103]:
print(data_y[0])
print(data_x[0])
print(labels_to_keep[0])

bygg
811555762115601/2018
811555762115601/2018


In [106]:
def create_bags(input_data, input_labels, positive_class, bag_count, instance_count):
    bags = []
    bag_labels = []

    # input_data = np.divide(input_data, 255.0)

    count = 0
    for _ in tqdm(range(bag_count), desc=f'Creating bag for {positive_class}'):
        index = np.random.choice(input_data.shape[0], instance_count, replace=False)
        instances_data = input_data[index]
        instances_labels = input_labels[index]

        bag_label = 0

        if positive_class in instances_labels:
            bag_label = 1
            count += 1

        bags.append(instances_data)
        bag_labels.append(np.array([bag_label]))

    print(f"Positive bags: {count}")
    print(f"Negative bags: {bag_count - count}")
    # return list(np.swapaxes(bags, 0, 1)), np.array(bag_labels)
    return np.array(bags), np.array(bag_labels)


train_data, train_labels = create_bags(np.array(x_train), np.array(y_train), 'bygg', BAG_COUNT, BAG_SIZE)

val_data, val_labels = create_bags(np.array(x_val), np.array(y_val), 'bygg', VAL_BAG_COUNT, BAG_SIZE)

Creating bag for bygg: 100%|██████████| 3500/3500 [00:07<00:00, 480.09it/s]


Positive bags: 3432
Negative bags: 68


Creating bag for bygg: 100%|██████████| 500/500 [00:00<00:00, 1168.42it/s]

Positive bags: 492
Negative bags: 8


In [36]:
def plot(data, labels, bag_class, predictions=None, attention_weights=None):

    labels = np.array(labels).reshape(-1)

    if bag_class == "positive":
        if predictions is not None:
            labels = np.where(predictions.argmax(1) == 1)[0]
            bags = np.array(data)[:, labels[0:PLOT_SIZE]]

        else:
            labels = np.where(labels == 1)[0]
            bags = np.array(data)[:, labels[0:PLOT_SIZE]]

    elif bag_class == "negative":
        if predictions is not None:
            labels = np.where(predictions.argmax(1) == 0)[0]
            bags = np.array(data)[:, labels[0:PLOT_SIZE]]
        else:
            labels = np.where(labels == 0)[0]
            bags = np.array(data)[:, labels[0:PLOT_SIZE]]

    else:
        print(f"There is no class {bag_class}")
        return

    print(f"The bag class label is {bag_class}")
    for i in range(PLOT_SIZE):
        # figure = plt.figure(figsize=(8, 8))
        print(f"Bag number: {labels[i]}")
        for j in range(BAG_SIZE):
            # image = bags[j][i]
            # figure.add_subplot(1, BAG_SIZE, j + 1)
            # plt.grid(False)
            # if attention_weights is not None:
            #     plt.title(np.around(attention_weights[labels[i]][j], 2))
            # plt.imshow(image)
            print(bags[j][i])
        # plt.show()


# Plot some of validation data bags per class.
plot(train_data, train_labels, "positive")
plot(train_data, train_labels, "negative")

The bag class label is positive
Bag number: 0
969198843/2017
982804264/2018
985257256/2018
990844011/2019
969089572/2019
977093880/2017
992725613/2019
976744314/2019
983064590/2019
969930404/2018
Bag number: 1
987668601/2018
969378612/2019
820802322/2018
985878684/2018
916463499/2018
969091879/2017
986639748/2018
969247682/2019
997736494/2017
969098326/2018
Bag number: 2
923088652/2019
969092670/2018
965208739/2019
977163048/2019
887932522/2018
991853812/2017
971156694/2019
987615931/2017
970569065/2018
980388565/2018


IndexError: index 97 is out of bounds for axis 1 with size 10

In [118]:

def train_generator():
    for i, row in enumerate(train_data):
        all_imgs = []
        for key in tqdm(row, total=len(row)):
            imgs = sd.get_images(key.split('/')[0], key.split('/')[1])
            all_imgs.append(imgs)

        yield all_imgs, train_labels[i]


def val_generator():
    for i, row in enumerate(val_data):
        all_imgs = []
        for key in tqdm(row, total=len(row)):

            imgs = sd.get_images(int(key.split('/')[0]), int(key.split('/')[1]))
            all_imgs.append(imgs)
        print(np.array(all_imgs[0]).shape)

        yield all_imgs, val_labels[i]

i = 0
for test in train_generator():
    print(np.array(test[0]).shape)
    print(test[1])
    if i > 5:
        break
    i += 1


train_dataset = tf.data.Dataset.from_generator(
    train_generator,
    output_types=(tf.dtypes.float64, tf.dtypes.int64),
    output_shapes=((7, 30, 16, 16, 12), 1)
)

val_dataset = tf.data.Dataset.from_generator(
    val_generator,
    output_types=(tf.dtypes.float64, tf.dtypes.int64),
    output_shapes=((7, 30, 16, 16, 12), 1)
)

# print(train_dataset)
# print(tuple(train_dataset))
# for test in train_generator():
#     print(np.array(test[0]).shape)
#     break

100%|██████████| 7/7 [00:00<00:00, 116.31it/s]


(7, 30, 16, 16, 12)
[1]


100%|██████████| 7/7 [00:00<00:00, 132.42it/s]


(7, 30, 16, 16, 12)
[1]


100%|██████████| 7/7 [00:00<00:00, 133.65it/s]


...

(7, 30, 16, 16, 12)
[1]


100%|██████████| 7/7 [00:00<00:00, 134.98it/s]


(7, 30, 16, 16, 12)
[1]


100%|██████████| 7/7 [00:00<00:00, 129.96it/s]


(7, 30, 16, 16, 12)
[1]


In [124]:
train_dataset.batch(2)


<BatchDataset element_spec=(TensorSpec(shape=(None, 7, 30, 16, 16, 12), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>

In [113]:
class MILAttentionLayer(layers.Layer):

    def __init__(
        self,
        weight_params_dim,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        use_gated=False,
        **kwargs,
    ):

        super().__init__(**kwargs)

        self.weight_params_dim = weight_params_dim
        self.use_gated = use_gated

        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

        self.v_init = self.kernel_initializer
        self.w_init = self.kernel_initializer
        self.u_init = self.kernel_initializer

        self.v_regularizer = self.kernel_regularizer
        self.w_regularizer = self.kernel_regularizer
        self.u_regularizer = self.kernel_regularizer


        # Input shape.
        # List of 2D tensors with shape: (batch_size, input_dim).
        input_dim = 16

        self.v_weight_params = self.add_weight(
            shape=(input_dim, self.weight_params_dim),
            initializer=self.v_init,
            name="v",
            regularizer=self.v_regularizer,
            trainable=True,
        )

        self.w_weight_params = self.add_weight(
            shape=(self.weight_params_dim, 1),
            initializer=self.w_init,
            name="w",
            regularizer=self.w_regularizer,
            trainable=True,
        )

        if self.use_gated:
            self.u_weight_params = self.add_weight(
                shape=(input_dim, self.weight_params_dim),
                initializer=self.u_init,
                name="u",
                regularizer=self.u_regularizer,
                trainable=True,
            )
        else:
            self.u_weight_params = None

        self.input_built = True

    def __call__(self, inputs):

        # Assigning variables from the number of inputs.
        instances = [self.compute_attention_scores(instance) for instance in inputs]

        # Apply softmax over instances such that the output summation is equal to 1.
        alpha = tf.math.softmax(instances, axis=0)

        return [alpha[i] for i in range(alpha.shape[0])]

    def compute_attention_scores(self, instance):

        # Reserve in-case "gated mechanism" used.
        original_instance = instance

        # tanh(v*h_k^T)
        instance = tf.math.tanh(tf.tensordot(instance, self.v_weight_params, axes=1))

        # for learning non-linear relations efficiently.
        if self.use_gated:

            instance = instance * tf.math.sigmoid(
                tf.tensordot(original_instance, self.u_weight_params, axes=1)
            )

        # w^T*(tanh(v*h_k^T)) / w^T*(tanh(v*h_k^T)*sigmoid(u*h_k^T))
        return tf.tensordot(instance, self.w_weight_params, axes=1)

In [114]:
def create_model(instance_shape):

    # Extract features from inputs.
    inputs, embeddings = [], []
    shared_dense_layer_1 = layers.Dense(128, activation="relu")
    shared_dense_layer_2 = layers.Dense(64, activation="relu")
    for _ in range(BAG_SIZE):
        inp = layers.Input(instance_shape)
        flatten = layers.Flatten()(inp)
        dense_1 = shared_dense_layer_1(flatten)
        dense_2 = shared_dense_layer_2(dense_1)
        inputs.append(inp)
        embeddings.append(dense_2)

    # Invoke the attention layer.
    alpha = MILAttentionLayer(weight_params_dim=256, kernel_regularizer=keras.regularizers.l2(0.01), use_gated=True, name="alpha")(embeddings)

    # Multiply attention weights with the input layers.
    multiply_layers = [
        layers.multiply([alpha[i], embeddings[i]]) for i in range(len(alpha))
    ]

    # Concatenate layers.
    concat = layers.concatenate(multiply_layers, axis=1)

    # Classification output node.
    output = layers.Dense(2, activation="softmax")(concat)

    return keras.Model(inputs, output)

In [115]:
def compute_class_weights(labels):

    # Count number of postive and negative bags.
    negative_count = len(np.where(labels == 0)[0])
    positive_count = len(np.where(labels == 1)[0])
    total_count = negative_count + positive_count

    # Build class weight dictionary.
    return {
        0: (1 / negative_count) * (total_count / 2),
        1: (1 / positive_count) * (total_count / 2),
    }

In [129]:
def train(train_data, train_labels, val_data, val_labels, model):

    # Take the file name from the wrapper.
    file_path = "/tmp/best_model_weights.h5"

    # Initialize model checkpoint callback.
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        file_path,
        monitor="val_loss",
        verbose=0,
        mode="min",
        save_best_only=True,
        save_weights_only=True,
    )

    early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, mode="min")

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=50,
        class_weight=compute_class_weights(train_labels),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    model.load_weights(file_path)

    return model


instance_shape = train_data[0][0].shape
print(instance_shape)
model = create_model((30, 16, 16, 12))

print(model.summary())

trained_model = train(train_data, train_labels, val_data, val_labels, model)

()
Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_50 (InputLayer)          [(None, 30, 16, 16,  0           []                               
                                 12)]                                                             
                                                                                                  
 input_51 (InputLayer)          [(None, 30, 16, 16,  0           []                               
                                 12)]                                                             
                                                                                                  
 input_52 (InputLayer)          [(None, 30, 16, 16,  0           []                               
                                 12)]                                                    

ValueError: in user code:

    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "E:\OneDrive - Universitetet i Agder\kornmo-master-thesis\.venv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtere

In [163]:
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()

In [166]:
x_train.shape

(60000, 28, 28)